# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,saskylakh,99,RU,1569467805,76,71.97,114.09,35.51,7.63
1,1,albany,40,US,1569467666,67,42.65,-73.75,62.60,4.70
2,2,upernavik,100,GL,1569467805,64,72.79,-56.15,33.35,11.48
3,3,hilo,1,US,1569467765,62,19.71,-155.08,84.20,5.82
4,4,torbay,90,CA,1569467701,100,47.66,-52.73,55.00,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat","Lng"]]

# Filla NaN values
locations = locations.dropna()
locations.head()


,Lat,Lng
0,71.97,114.09
1,42.65,-73.75
2,72.79,-56.15
3,19.71,-155.08
4,47.66,-52.73


In [13]:
# Defining Humidity as weight
weights = weather_df["Humidity"].astype(float)


In [17]:
#Set up your map
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
# Narrow new DF to the 3 criterias defined for the exercise
weather2 = weather_df
weather2 = weather2[(weather2["Max Temp"] > 70)]
weather2 = weather2[(weather2["Max Temp"] < 80)] 
weather2 = weather2[(weather2["Wind Speed"] < 10)]
weather2 = weather2[(weather2["Cloudiness"] == 0)]

# drop null
weather2 = weather2.dropna()
weather2.count()

Unnamed: 0    17
City          17
Cloudiness    17
Country       17
Date          17
Humidity      17
Lat           17
Lng           17
Max Temp      17
Wind Speed    17
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
# Store into variable named hotel_df.
hotel_df = weather2

# set up additional column to hold information
hotel_df['Hotel Name'] = " "
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,13,rikitea,0,PF,1569467808,68,-23.12,-134.97,72.2,8.59,
150,150,parkes,0,AU,1569467776,22,-33.14,148.18,75.0,6.71,
174,174,benjamin aceval,0,PY,1569467850,40,-24.97,-57.57,71.6,5.82,
202,202,alice springs,0,AU,1569467855,33,-23.70,133.88,75.2,5.82,
215,215,iquitos,0,PE,1569467858,94,-3.75,-73.25,75.2,1.07,


In [83]:
# find the closest hotel of each type to coordinates
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # convert to json
    data = response.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [86]:
hotel_df2 = hotel_df
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,13,rikitea,0,PF,1569467808,68,-23.12,-134.97,72.20,8.59,Pension Bianca & Benoit
150,150,parkes,0,AU,1569467776,22,-33.14,148.18,75.00,6.71,Hotel Gracelands
174,174,benjamin aceval,0,PY,1569467850,40,-24.97,-57.57,71.60,5.82,Escuela Agrícola y Hotel Cerrito
202,202,alice springs,0,AU,1569467855,33,-23.70,133.88,75.20,5.82,DoubleTree by Hilton Hotel Alice Springs
215,215,iquitos,0,PE,1569467858,94,-3.75,-73.25,75.20,1.07,Hotel Dorado
227,227,cape town,0,ZA,1569467861,51,-33.93,18.42,72.00,8.05,Check Inn Hotel
257,257,timizart,0,DZ,1569467868,75,36.80,4.25,74.16,6.44,
267,267,lasa,0,CY,1569467871,83,34.92,32.53,71.60,6.93,Lasa Heights Hotel
292,292,baghdad,0,IQ,1569467653,34,33.30,44.38,78.80,5.82,Baghdad Hotel
304,304,shizilu,0,CN,1569467876,52,29.63,109.09,75.33,0.40,Langyi Hotel


In [93]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout={
        'padding': '3px',
        'border': '1px solid black'
})
locations2 = hotel_df[["Lat", "Lng"]]
fig.add_layer(gmaps.heatmap_layer(locations2))
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px'))